In [52]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [53]:
# Load the labeled dataset
df = pd.read_csv(r"C:\Users\ISMAIL\OneDrive\Documents\data science projects\reviews_sentiment.csv")
df.head()

,Product_name,Review,Sentiment,Rating
0,Kindle Paperwhite,initially trouble deciding paperwhite voyage r...,positive,5.0
1,Kindle Paperwhite,allow preface little history casual reader own...,positive,5.0
2,Kindle Paperwhite,enjoying far great reading original fire since...,positive,4.0
3,Kindle Paperwhite,bought one first paperwhites pleased constant ...,positive,5.0
4,Kindle Paperwhite,say upfront dont like coroporate hermetically ...,positive,5.0


In [54]:
from sklearn.model_selection import train_test_split

# Remove rows with missing values
df = df.dropna()

# Extract the texts and labels
texts = df["Product_name"] + " " + df["Review"]
labels = df["Sentiment"]

# Split the data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [56]:
import pandas as pd

# Assuming your data is in a DataFrame called 'df' with a 'Sentiment' column
class_distribution = df['Sentiment'].value_counts()
print(class_distribution)

Sentiment
positive    977
negative    495
neutral     124
Name: count, dtype: int64


In [73]:
import nlpaug.augmenter.word as naw
import random
# Define an augmentation function
def augment_text(text):
    aug = naw.SynonymAug(aug_p=0.3)  # Example: Synonym augmentation
    augmented_text = aug.augment(text)
    return augmented_text

# Apply data augmentation to the training dataset
augmented_train_texts = [augment_text(text) for text in train_texts]

# Combine the augmented texts with the original training data
augmented_train_texts.extend(train_texts)
augmented_train_labels = np.concatenate([train_labels, train_labels])

# Shuffle the augmented training data
augmented_train_data = list(zip(augmented_train_texts, augmented_train_labels))
random.shuffle(augmented_train_data)
augmented_train_texts, augmented_train_labels = zip(*augmented_train_data)

print("Augmented texts:")
for text in augmented_train_texts:
    print(text)

print("Augmented labels:")
for label in augmented_train_labels:
    print(label)


Augmented texts:
Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker love bought mother father easy use
['Amazon Premium Headphones ive purchased items amazon years rarely publish reviews felt compelled write one although reviews point never seen good ware get bad reviews im hoping people purchase star evaluation accurate product ill address three biggest complaints ive read otherssound quality think headphones sound great ive never owned expensive duo im able compare higherend offerings ive owned dozens cheap midrange headphones range im audiophile musician im missing something coming isnt im fine ignorance would say theyre slightly better apples buds liked much used years recently using redesigned ones year theres tinny listening sound thats alltoocommon headphones price range far people saying dont compare apples buds well dont know maybe crazy exclusive interlingual rendition apples buds amazing compared regular earpods think amazons headphones actually sound bit good ply nice fu

In [68]:
#!pip install nlpaug



     ------------------------------------ 410.5/410.5 kB 213.5 kB/s eta 0:00:00


In [79]:
# Define the number of classes
num_classes = 3  # Update this value based on your classification problem

# Define and compile the LSTM model
model_augmented = Sequential()
model_augmented.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model_augmented.add(LSTM(units=128, dropout=0.2))
model_augmented.add(Dense(num_classes, activation='softmax'))
model_augmented.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model using augmented data
model_augmented.fit(train_data_augmented, train_labels_augmented, 
                    validation_data=(val_data_augmented, val_labels_augmented),
                    epochs=5, batch_size=64)

# Evaluate the model on the original test set
test_data_encoded = tokenizer.texts_to_sequences(test_texts)
test_data_padded = pad_sequences(test_data_encoded, maxlen=max_length, padding='post')
test_labels_encoded = label_encoder.transform(test_labels)
test_labels_one_hot = to_categorical(test_labels_encoded, num_classes=num_classes)
loss, accuracy = model_augmented.evaluate(test_data_padded, test_labels_one_hot)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/5
32/32 [==============================] - 14s 279ms/step - loss: 0.8871 - accuracy: 0.6173 - val_loss: 0.8671 - val_accuracy: 0.6047
Epoch 2/5
32/32 [==============================] - 8s 261ms/step - loss: 0.8247 - accuracy: 0.6281 - val_loss: 0.8608 - val_accuracy: 0.6067
Epoch 3/5
32/32 [==============================] - 7s 224ms/step - loss: 0.8049 - accuracy: 0.6521 - val_loss: 0.8482 - val_accuracy: 0.6282
Epoch 4/5
32/32 [==============================] - 7s 233ms/step - loss: 0.7843 - accuracy: 0.6644 - val_loss: 0.8420 - val_accuracy: 0.6341
Epoch 5/5
10/10 [==============================] - 1s 58ms/step - loss: 0.7652 - accuracy: 0.6875
Test Loss: 0.7651818990707397
Test Accuracy: 0.6875


In [81]:
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold

# Define the maximum length for padding sequences
max_length = 100

# Convert augmented texts to numerical representations
train_data_augmented = tokenizer.texts_to_sequences(augmented_train_texts)

# Pad sequences to ensure consistent length
train_data_augmented = pad_sequences(train_data_augmented, maxlen=max_length, padding='post')

# Convert augmented labels to one-hot encoding
train_labels_augmented = to_categorical(augmented_train_labels)

# Define the vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Define the number of classes
num_classes = 3  # Update this value based on your classification problem

# Define and compile the LSTM model
model_augmented = Sequential()
model_augmented.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model_augmented.add(LSTM(units=128, dropout=0.2))
model_augmented.add(Dense(num_classes, activation='softmax'))
model_augmented.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Perform cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_scores = []
for train_index, val_index in kfold.split(train_data_augmented, augmented_train_labels):
    # Split augmented data into training and validation sets for the current fold
    X_train, X_val = train_data_augmented[train_index], train_data_augmented[val_index]
    y_train, y_val = train_labels_augmented[train_index], train_labels_augmented[val_index]

    # Train the model on the current fold
    model_augmented.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=5, batch_size=64)

    # Evaluate the model on the current fold
    loss, accuracy = model_augmented.evaluate(X_val, y_val)
    fold_scores.append(accuracy)

# Calculate the average accuracy across all folds
average_accuracy = sum(fold_scores) / len(fold_scores)
print("Average Accuracy:", average_accuracy)

# Evaluate the model on the original test set
test_data_encoded


Epoch 1/5
32/32 [==============================] - 8s 176ms/step - loss: 0.8822 - accuracy: 0.6090 - val_loss: 0.8420 - val_accuracy: 0.6184
Epoch 2/5
32/32 [==============================] - 6s 175ms/step - loss: 0.8288 - accuracy: 0.6330 - val_loss: 0.8427 - val_accuracy: 0.6164
Epoch 3/5
32/32 [==============================] - 5s 169ms/step - loss: 0.8025 - accuracy: 0.6629 - val_loss: 0.8635 - val_accuracy: 0.6125
Epoch 4/5
32/32 [==============================] - 6s 180ms/step - loss: 0.7857 - accuracy: 0.6693 - val_loss: 0.8461 - val_accuracy: 0.6106
Epoch 5/5
16/16 [==============================] - 1s 51ms/step - loss: 0.8729 - accuracy: 0.6067
Epoch 1/5
32/32 [==============================] - 8s 251ms/step - loss: 0.7894 - accuracy: 0.6610 - val_loss: 0.7506 - val_accuracy: 0.6810
Epoch 2/5
32/32 [==============================] - 8s 252ms/step - loss: 0.7743 - accuracy: 0.6659 - val_loss: 0.7636 - val_accuracy: 0.6810
Epoch 3/5
32/32 [==============================] - 7s 20

[[1, 5, 10, 28, 24, 23, 14, 43, 803, 135, 2352, 130, 57, 2546],
 [1, 5, 10, 28, 24, 23, 14, 5, 116, 24, 502, 1, 33],
 [1,
  5,
  10,
  28,
  24,
  23,
  14,
  203,
  856,
  16,
  4387,
  127,
  1636,
  4,
  443,
  163,
  130,
  57,
  1544,
  29,
  36,
  2107,
  101,
  1122,
  236],
 [1,
  2,
  6,
  655,
  15,
  1943,
  50,
  1612,
  370,
  851,
  43,
  1,
  1049,
  173,
  138,
  338,
  729,
  2,
  6,
  1117,
  792,
  386,
  1241,
  54,
  449,
  557,
  4744,
  1088,
  297,
  68,
  1250,
  1730,
  17,
  147,
  1877,
  521,
  1281,
  229,
  78,
  16,
  310,
  65,
  252,
  2843,
  766,
  587,
  43,
  22,
  247,
  2257,
  1425,
  163,
  549,
  564,
  549,
  386,
  15,
  183,
  712,
  1587,
  2738,
  122,
  740,
  1312,
  642,
  18,
  1272,
  646,
  457,
  276,
  1455,
  1109,
  6577,
  1821,
  258,
  970,
  702,
  44,
  1729,
  258,
  1312,
  642,
  17,
  3451,
  521,
  247,
  2257,
  173,
  200,
  36,
  794,
  3809,
  647,
  6,
  118,
  647,
  6,
  363,
  1884,
  2870,
  6305,
  879,
  587

In [83]:
from sklearn.metrics import classification_report, confusion_matrix

# Make predictions on the test set
test_predictions = model_augmented.predict(test_data_padded)  # Replace 'test_data' with your own test data
test_predictions = np.argmax(test_predictions, axis=1)
test_true_labels = np.argmax(test_labels_one_hot, axis=1)  # Replace 'test_labels' with your own test labels

# Classification report
print("Classification Report:")
print(classification_report(test_true_labels, test_predictions))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(test_true_labels, test_predictions))


10/10 [==============================] - 4s 66ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.38      0.44        92
           1       0.53      0.30      0.38        27
           2       0.72      0.86      0.78       201

    accuracy                           0.67       320
   macro avg       0.59      0.51      0.54       320
weighted avg       0.65      0.67      0.65       320

Confusion Matrix:
[[ 35   6  51]
 [  3   8  16]
 [ 28   1 172]]


In [84]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Define a function to build and compile the model
def build_model(num_lstm_units, dropout_rate):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
    model.add(LSTM(units=num_lstm_units, dropout=dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define the hyperparameters to tune
param_grid = {
    'num_lstm_units': [64, 128, 256],
    'dropout_rate': [0.2, 0.3, 0.4]
}

# Create a KerasClassifier based on the build_model function
model = KerasClassifier(build_fn=build_model, epochs=5, batch_size=64)

# Perform grid search with cross-validation
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_data_augmented, train_labels_augmented)

# Print the best hyperparameters and validation accuracy
print("Best Hyperparameters: ", grid_result.best_params_)
print("Best Validation Accuracy: ", grid_result.best_score_)

# Build and train the final model with the best hyperparameters
best_model = build_model(num_lstm_units=grid_result.best_params_['num_lstm_units'],
                         dropout_rate=grid_result.best_params_['dropout_rate'])
best_model.fit(train_data_augmented, train_labels_augmented,
               validation_data=(val_data_augmented, val_labels_augmented),
               epochs=5, batch_size=64)

# Evaluate the final model on the original test set
loss, accuracy = best_model.evaluate(test_data_padded, test_labels_one_hot)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


C:\Users\ISMAIL\AppData\Local\Temp\ipykernel_6260\514977092.py:20: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=build_model, epochs=5, batch_size=64)


Epoch 1/5
27/27 [==============================] - 8s 133ms/step - loss: 0.9211 - accuracy: 0.5979
Epoch 2/5
27/27 [==============================] - 3s 123ms/step - loss: 0.8301 - accuracy: 0.6320
Epoch 3/5
27/27 [==============================] - 3s 122ms/step - loss: 0.8139 - accuracy: 0.6408
Epoch 4/5
27/27 [==============================] - 3s 123ms/step - loss: 0.7944 - accuracy: 0.6673
Epoch 5/5
14/14 [==============================] - 2s 45ms/step - loss: 0.8324 - accuracy: 0.6240
Epoch 1/5
27/27 [==============================] - 7s 124ms/step - loss: 0.9271 - accuracy: 0.5885
Epoch 2/5
27/27 [==============================] - 3s 115ms/step - loss: 0.8459 - accuracy: 0.6102
Epoch 3/5
27/27 [==============================] - 3s 116ms/step - loss: 0.8278 - accuracy: 0.6238
Epoch 4/5
27/27 [==============================] - 4s 144ms/step - loss: 0.8133 - accuracy: 0.6502
Epoch 5/5
14/14 [==============================] - 2s 44ms/step - loss: 0.7924 - accuracy: 0.6710
Epoch 1/5
27

Epoch 1/5
27/27 [==============================] - 16s 452ms/step - loss: 0.9095 - accuracy: 0.6011
Epoch 2/5
27/27 [==============================] - 14s 520ms/step - loss: 0.8336 - accuracy: 0.6228
Epoch 3/5
27/27 [==============================] - 14s 503ms/step - loss: 0.8194 - accuracy: 0.6240
Epoch 4/5
27/27 [==============================] - 14s 515ms/step - loss: 0.7983 - accuracy: 0.6481
Epoch 5/5
14/14 [==============================] - 4s 233ms/step - loss: 0.8247 - accuracy: 0.6365
Epoch 1/5
27/27 [==============================] - 35s 1s/step - loss: 0.8807 - accuracy: 0.6179
Epoch 2/5
27/27 [==============================] - 31s 1s/step - loss: 0.8268 - accuracy: 0.6320
Epoch 3/5
27/27 [==============================] - 31s 1s/step - loss: 0.8113 - accuracy: 0.6514
Epoch 4/5
27/27 [==============================] - 32s 1s/step - loss: 0.7933 - accuracy: 0.6661
Epoch 5/5
14/14 [==============================] - 5s 338ms/step - loss: 0.8402 - accuracy: 0.6240
Epoch 1/5
27/2

40/40 [==============================] - 13s 230ms/step - loss: 0.8796 - accuracy: 0.6070 - val_loss: 0.8656 - val_accuracy: 0.6047
Epoch 2/5
40/40 [==============================] - 8s 203ms/step - loss: 0.8267 - accuracy: 0.6281 - val_loss: 0.8460 - val_accuracy: 0.6438
Epoch 3/5
40/40 [==============================] - 8s 200ms/step - loss: 0.8058 - accuracy: 0.6583 - val_loss: 0.8168 - val_accuracy: 0.6517
Epoch 4/5
40/40 [==============================] - 8s 204ms/step - loss: 0.7894 - accuracy: 0.6571 - val_loss: 0.8011 - val_accuracy: 0.6556
Epoch 5/5
10/10 [==============================] - 1s 65ms/step - loss: 0.7380 - accuracy: 0.6719
Test Loss: 0.7379600405693054
Test Accuracy: 0.671875


In [2]:
#misclassified_examples = []

#for i, (text, true_label) in enumerate(zip(test_texts, test_labels)):
    #predicted_label = label_encoder.inverse_transform([np.argmax(predictions[i])])[0]

    #if predicted_label != true_label:
        #misclassified_examples.append((text, true_label, predicted_label))

# Print the misclassified examples
#for example in misclassified_examples:
    #print("Text: ", example[0])
    #print("True Label: ", example[1])
    #print("Predicted Label: ", example[2])
    #print("---")


In [1]:
#for example in misclassified_examples:
    #text = example[0]
    #true_label = example[1]
   # predicted_label = example[2]
    # Perform analysis on the misclassified example, such as printing the text, true label, and predicted label
    #print("Text:", text)
    #print("True Label:", true_label)
   # print("Predicted Label:", predicted_label)
   # print()


In [3]:
#positive_label = "True Label"
#negative_label = "Predicted Label"


#false_positives = []
#false_negatives = []

#for example in misclassified_examples:
    #text = example[0]
    #true_label = example[1]
    #predicted_label = example[2]
    
    #if true_label == positive_label and predicted_label == negative_label:
        #false_negatives.append(example)
    #elif true_label == negative_label and predicted_label == positive_label:
        #false_positives.append(example)
    
# Perform analysis on false positives and false negatives
# Print the texts and labels of the false positives and false negatives


# Print the texts and labels of the false positives
#print("False Positives:")
#for example in false_positives:
    #text = example[0]
    #true_label = example[1]
    #predicted_label = example[2]
    #print("Text:", text)
    #print("True Label:", true_label)
    #print("Predicted Label:", predicted_label)
    #print()

# Print the texts and labels of the false negatives
#print("False Negatives:")
#for example in false_negatives:
    #text = example[0]
    #true_label = example[1]
    #predicted_label = example[2]
    #print("Text:", text)
    #print("True Label:", true_label)
    #print("Predicted Label:", predicted_label)
    #print()
